# Importance to use the Trailed Source Magnitude For Your Analysis of Sorcha Output

Sorcha computes two magnitudes for each potential LSST discovery/detection chance: the trailed source magnitude and the PSF magnitude. The PSF magnitude is the apparent magnitude in a given filter measured by the data management pipelines assuming a stellar-like PSF. The trailed source magnitude is apparent magnitude in a given filter adding up all of the counts in the trail (as the simulated object is moving object). The PSF magnitude is what is used in detection by the Rubin transient soure detection pipelines, but the trailed source magnitude is the true apparent megnitude of the moving object.  

Sorcha can output both magnitudes, but by default will output only the trailed source magnitude. Other option options may output both values.  One must be careful use the correct apparent magnitude in your analysis. For anything invovling photometry (and not detectability), the recommend is to use the trailed soruce manigude. In this notebook, we show why it is important to use the trailed source magnitude.

Let's demonstrate the combined effect of including both the phase curve and the trailing loss for a main belt asteroid. 

This will using functionality from both the `demo_ApparentMagnitudeValidation` and `demo_TrailingLossesValidation` notebooks. 

In [ ]:
import pandas as pd
import numpy as np
import astropy.units as u
from astroquery.jplhorizons import Horizons
from sorcha.modules.PPCalculateApparentMagnitudeInFilter import PPCalculateApparentMagnitudeInFilter
from sorcha.modules.PPTrailingLoss import PPTrailingLoss
import matplotlib.pyplot as plt

Let's grab two years of positions for asteroid Themis as expected from the Rubin Obervatory site (Obs Code X05)

In [ ]:
obj = Horizons(id='Themis', id_type='name', location='X05',

               epochs={'start':'2021-01-01', 'stop':'2023-01-01',

                       'step':'1d'})

eph = obj.ephemerides(quantities='1,3,9,19,20,43')
jpl_eph = eph.to_pandas()
jpl_eph

In [ ]:
observations_df = pd.DataFrame({'MJD':jpl_eph['datetime_jd'] - 2_400_000.5,
                                'H_filter': jpl_eph['H'],
                                'GS': jpl_eph['G'],
                                'Dec_deg' : np.zeros_like(jpl_eph['G']),
                                'G1': np.zeros(len(jpl_eph['G'])) + 0.62,
                                'G2': np.zeros(len(jpl_eph['G'])) + 0.14,
                                'G12': np.zeros(len(jpl_eph['G'])) + 0.68,
                                'JPL_mag': jpl_eph['V'],
                                'Range_LTC_km': jpl_eph['r'] * 1.495978707e8, #au to km
                                'Obj_Sun_LTC_km': jpl_eph['delta'] * 1.495978707e8,
                                'phase_deg': jpl_eph['alpha_true'],
                                'RARateCosDec_deg_day' : jpl_eph['RA_rate'] * 24/3600, # note horizons already applies the cos(dec) term 
                                'DecRate_deg_day' : jpl_eph['DEC_rate']* 24/3600,
                                'seeingFwhmEff_arcsec' : 0.8 * np.ones_like(jpl_eph['G']),
                                'visitExposureTime' : 30.0 * np.ones_like(jpl_eph['G']),
                                })

Calculate the PSF Magnitude and Trailed Source Magnitude for these observations including the  Bowell et al. (1989) HG phase curve

In [ ]:
observations_df = PPCalculateApparentMagnitudeInFilter(observations_df, 'HG', 'r', 'HG_mag')
dmagDetect = PPTrailingLoss(observations_df, "circularPSF")
observations_df['trailedMagnitude'] = observations_df['HG_mag'] + dmagDetect


Let's plot up the results to show that there is a difference between the phase curve you will get if you use 

In [ ]:
plt.plot(observations_df['phase_deg'], observations_df['trailedMagnitude']- 5 * np.log10(observations_df['Range_LTC_km'] * observations_df['Obj_Sun_LTC_km']/(1.495978707e8**2)), '.', label='PSF-assumed magnitude')
plt.plot(observations_df['phase_deg'], observations_df['HG_mag'] - 5 * np.log10(observations_df['Range_LTC_km'] * observations_df['Obj_Sun_LTC_km']/(1.495978707e8**2)), '-', label='Trail-corrected magnitude')

plt.xlabel('Phase angle (deg)')
plt.ylabel('Reduced magnitude')
plt.legend(loc=3)
plt.gca().invert_yaxis()
plt.show()

Using the wrong value will produce an offset in the recovered coefficients of the phase function (both $H$ and $G$). To show this, we will use an independent implementation (see the `demo_ApparentMagnitudeValidation` notebook) of the HG phase curve model and do a simplified fit the $G$ coefficient in both cases.

In [ ]:
A = [3.332, 1.862]
B = [0.631, 1.218]
C = [0.986, 0.238]

def HG_model(phase, params):
    sin_a = np.sin(phase)
    tan_ah = np.tan(phase/2)
    
    W = np.exp(-90.56 * tan_ah * tan_ah)    
    scale_sina = sin_a/(0.119 + 1.341*sin_a - 0.754*sin_a*sin_a)
    
    phi_1_S = 1 - C[0] * scale_sina
    phi_2_S = 1 - C[1] * scale_sina
    
    phi_1_L = np.exp(-A[0] * np.power(tan_ah, B[0]))
    phi_2_L = np.exp(-A[1] * np.power(tan_ah, B[1]))
    
    phi_1 = W * phi_1_S + (1-W) * phi_1_L
    phi_2 = W * phi_2_S + (1-W) * phi_2_L
    return params[0] - 2.5*np.log10((1-params[1])* phi_1 + (params[1]) * phi_2) 

def chi2(params, mag, phase):
    pred = HG_model(phase, params)
    return (mag - pred)

def fit(mag, phase, params=[0.1]):
    from scipy.optimize import leastsq
    phase = np.deg2rad(phase)


    sol = leastsq(chi2, [mag[0]] + params,  (mag, phase), full_output = True)

    return sol[0]



Phase curve parameters from fitting for the reduced magnitude calculated from the PSF magnitude (which does not account for the source potentially having an extended PSF)

In [ ]:
solution = fit(observations_df['HG_mag']- 5 * np.log10(observations_df['Range_LTC_km'] * observations_df['Obj_Sun_LTC_km']/(1.495978707e8**2)), observations_df['phase_deg'])

print(f'H = {solution[0]}, G = {solution[1]}')

Phase curve parameters from fitting for the reduced magnitude calculated from the trailed source magnitude (which does account for the source being trailed due to motion of the simulated moving object during the ~30s LSST exposure.. 

In [ ]:
solution = fit(observations_df['trailedMagnitude']- 5 * np.log10(observations_df['Range_LTC_km'] * observations_df['Obj_Sun_LTC_km']/(1.495978707e8**2)), observations_df['phase_deg'])

print(f'H = {solution[0]}, G = {solution[1]}')

Both the estimated H and phase curve parameters will be offset if one incorrectly uses the PSF mag instead of the trailed soure mag. 